# Amazon reviews classification [sklearn]
Binary classification of product's review 'helpfulness' (quality).
Reference: <https://t-lanigan.github.io/amazon-review-classifier/>

## Import libraries

In [1]:
import string

import giskard
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from giskard import wrap_dataset, wrap_model
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

## Notebook-level settings

In [2]:
# Disable chained assignment warning.
pd.options.mode.chained_assignment = None

## Define constants

In [3]:
# Constants.
RANDOM_SEED = 0
TEST_RATIO = 0.2

TARGET_THRESHOLD = 0.5
TARGET_NAME = "isHelpful"

# Paths.
DATASET_URL = 'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_5.json.gz'

## Load and preprocess Dataset

In [4]:
def download_data(**kwargs: dict) -> pd.DataFrame:
    """Download the dataset using URL."""
    print(f"Downloading dataset from {DATASET_URL}")

    _df = pd.read_json(DATASET_URL, compression="gzip", lines=True, **kwargs)

    print(f"Dataset was loaded!")

    return _df

In [5]:
def preprocess_data(_df: pd.DataFrame) -> pd.DataFrame:
    """Perform data-preprocessing steps."""
    print(f"Start data preprocessing...")

    # Select columns.
    _df = _df[["reviewText", "helpful"]]

    # Extract numbers of helpful and total votes.
    _df['helpful_ratings'] = _df.helpful.apply(lambda x: x[0])
    _df['total_ratings'] = _df.helpful.apply(lambda x: x[1])

    # Filter unreasonable comments.
    _df = _df[_df.total_ratings > 10]

    # Create target column.
    _df[TARGET_NAME] = np.where((_df.helpful_ratings / _df.total_ratings) > TARGET_THRESHOLD, 1, 0)

    # Delete columns we don't need anymore.
    _df.drop(columns=["helpful", 'helpful_ratings', 'total_ratings'], inplace=True)

    print("Data preprocessing finished!")

    return _df

In [6]:
reviews_df = download_data()
reviews_df = preprocess_data(reviews_df)
reviews_df

Dataset was loaded!
Start data preprocessing...
Data preprocessing finished!


,reviewText,isHelpful
33,"I did the 7-day trial and was impressed, but a...",1
94,"This app is great! It has a daily verse, a da...",1
284,This app simply latches on to Google maps and ...,0
288,"Unable to move to sd card, uninstalled. Cannot...",0
296,With all of the idiocy in almost every free ap...,1
...,...,...
752842,"Yes, it is! Love the sound effects. game grows...",1
752843,amazing. 1track but hard enough so it doesn't ...,1
752846,"I'd give it -5 stars if I could. I mean, I lov...",1
752914,This game is one of the most realistic dinosau...,1


## Train-test split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(reviews_df[["reviewText"]], reviews_df[TARGET_NAME],
                                                    test_size=TEST_RATIO, random_state=RANDOM_SEED)

## Wrap test dataset

In [8]:
test_dataset = pd.concat([X_test, y_test], axis=1)
wrapped_dataset = wrap_dataset(test_dataset,
                               name="reviews", target=TARGET_NAME, column_types={"reviewText": "text"})

Your 'pandas.DataFrame' dataset is successfully wrapped by Giskard's 'Dataset' wrapper class.


## Define preprocessing pipeline

In [9]:
def make_lowercase(x):
    """Lower an input string."""
    x = x.reviewText.apply(lambda row: row.lower())
    return x

def remove_punctuation(x):
    """Remove punctuation from input string."""
    x.apply(lambda row: row.translate(str.maketrans('', '', string.punctuation)))
    return x

stemmer = SnowballStemmer("english")
def tokenizer(x):
    """Define string tokenization logic."""
    x = x.split()
    stems = list()
    [stems.append(stemmer.stem(word)) for word in x]
    return stems

vectorizer = TfidfVectorizer(tokenizer=tokenizer, stop_words='english', ngram_range=(1, 1), min_df=0.01)

In [10]:
preprocessor = Pipeline(steps=[
    ("lowercase", FunctionTransformer(make_lowercase)),
    ("punctuation", FunctionTransformer(remove_punctuation)),
    ("vectorizer", vectorizer)
])

## Build estimator

In [11]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("estimator", LogisticRegression(random_state=RANDOM_SEED))
])

In [12]:
pipeline.fit(X_train, y_train)

/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('lowercase',
                                  FunctionTransformer(func=<function make_lowercase at 0x17bc496c0>)),
                                 ('punctuation',
                                  FunctionTransformer(func=<function remove_punctuation at 0x17bc49510>)),
                                 ('vectorizer',
                                  TfidfVectorizer(min_df=0.01,
                                                  stop_words='english',
                                                  tokenizer=<function tokenizer at 0x17bc49480>))])),
                ('estimator', LogisticRegression(random_state=0))])

In [13]:
train_metric = roc_auc_score(y_train, pipeline.predict_proba(X_train)[:, 1].T)
test_metric = roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1].T)

print(f"Train ROC-AUC score: {train_metric}")
print(f"Test ROC-AUC score: {test_metric}")

Train ROC-AUC score: 0.7602054998587102
Test ROC-AUC score: 0.7400220231290606


## Wrap model

In [14]:
wrapped_model = wrap_model(model=pipeline,
                           model_type="classification",
                           feature_names=["reviewText"],
                           name="review_helpfulness_predictor",
                           classification_threshold=0.5,
                           classification_labels=pipeline.classes_)

Your 'sklearn' model is successfully wrapped by Giskard's 'SKLearnModel' wrapper class.


## Scan model

In [15]:
results = giskard.scan(model=wrapped_model, dataset=wrapped_dataset)

/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/Users/mykytaalekseiev/IdeaProjects/GiskardExamples/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 't

Hint: "Your target variable values are numeric. It is recommended to have Human readable string as your target values to make results more understandable in Giskard."
Your model is successfully validated.


In [16]:
display(results)

<iframe srcdoc="
<script src="https://cdn.tailwindcss.com"></script>
<script>
 tailwind.config = {
 darkMode: 'class'
 }
</script>
<style>
 table.dataframe {
 width: 100%;
 max-width: 100%;
 overflow: auto;
 }

 .dataframe tr {
 border-bottom: 1px solid #555;
 }

 .dataframe td,
 .dataframe th {
 padding: 0.5rem;
 }
</style>



<div class="dark">
 <div id="gsk-scan" class="dark:text-white dark:bg-zinc-900">
 <!-- TAB HEADER -->
<div class="flex items-center items-stretch h-11">
 <div class="flex items-center px-4 dark:fill-white border-b border-gray-500 h-11">
 <svg xmlns="http://www.w3.org/2000/svg" xml:space="preserve" viewBox="0 0 410 213" height="15">
 <path
 d="M318.78 21.94a59.439 59.439 0 0 1 36.45-12.56v.03c7.09-.1 14.13 1.2 20.71 3.84 6.58 2.64 12.57 6.56 17.62 11.54s9.06 10.91 11.8 17.45a53.44 53.44 0 0 1 0 41.3 53.588 53.588 0 0 1-11.8 17.45c-5.05 4.98-11.04 8.9-17.62 11.54s-13.62 3.95-20.71 3.84l-67.92.16-35.87 49.11-135.24 46.84 40.81-47.24a139.229 139.229 0 0 1-66.06-23.8H45.11L0 101.46h55.73C60.38 44.84 115.97 0 183.59 0c31.71 0 63.58 9.86 87.47 27.04a105.149 105.149 0 0 1 26.65 27.18 59.4 59.4 0 0 1 21.07-32.28zM99.85 46.03l55.4 55.45h21.31l-64.81-64.87c-4.18 2.86-8.15 6.01-11.9 9.42zm57.78-28.45 83.9 83.9h21.29l-86.21-86.22c-6.37.32-12.71 1.1-18.98 2.32zm-92.84 98.94H39.72l11.1 9.82h22.44c-3-3.12-5.83-6.4-8.47-9.82zm134.27 0H84.8l-.03.01c20.8 20.91 51.79 33.67 84.66 34.3l29.63-34.31zm-34.81 63.35 77.79-26.94 26.6-36.39h-49.68l-54.71 63.33zm146.77-78.65h46.2l.05.01c9.98-.55 19.36-4.97 26.14-12.32a38.409 38.409 0 0 0 10.16-27.05 38.44 38.44 0 0 0-11.56-26.48 38.405 38.405 0 0 0-26.74-10.94A44.287 44.287 0 0 0 324 37.41a44.268 44.268 0 0 0-12.97 31.28v32.53zm51.82-52.83c5.27.52 9.12 5.22 8.6 10.49-.52 5.27-5.22 9.12-10.49 8.6s-9.12-5.22-8.6-10.49c.53-5.27 5.22-9.12 10.49-8.6z"
 style="fill-rule:evenodd;clip-rule:evenodd" />
 </svg>
 <div class="ml-4 py-2">
 <span class="uppercase text-sm">0 issues detected</span>
 </div>
 </div>

 

 <div class="flex-grow border-b border-gray-500 h-11"></div>
</div>
<!-- TAB HEADER END -->
 
 </div>
</div>
<script type="text/javascript">
 (function () {
 console.log("Loading Giskard Scan");
 let rows = document.querySelectorAll("#gsk-scan .gsk-issue")
 rows.forEach(rowEl => {
 {
 rowEl.addEventListener("click", (event) => {
 {
 event.preventDefault()
 if (event.target.classList.contains("gsk-debug")) {
 {
 alert("Not implemented yet")
 return;
 }
 }

 rowEl.classList.toggle("open")
 rowEl.classList.toggle("bg-zinc-700")
 }
 })
 }
 });

 const tabs = document.querySelectorAll("#gsk-scan [role='tabpanel']")
 const tabHeaders = document.querySelectorAll("#gsk-scan [data-tab-target]")
 tabHeaders.forEach(tabHeader => {
 tabHeader.addEventListener("click", (event) => {
 event.preventDefault()
 const tabId = tabHeader.getAttribute("data-tab-target")

 tabs.forEach(tab => {
 tab.classList.add("hidden")
 })
 tabHeaders.forEach(tabh => {
 tabh.classList.remove("active")
 })

 tabHeader.classList.add("active")
 console.log(tabId)
 document.getElementById(tabId).classList.remove("hidden")

 })
 })

 })()
</script>
" style="width: 100%; border: none;" class="gsk-scan">